# Term Graphs
> Term graphs that encode logic of rules as graphs over RA operators and ie function executions

In [ ]:
#| default_exp term_graph

In [ ]:
#| hide
from nbdev.showdoc import show_doc

%load_ext autoreload
%autoreload 2

In [ ]:
#| export

from IPython.display import display
import pandas as pd
import os
import networkx as nx
import itertools
import logging
import pytest
from collections import defaultdict
logger = logging.getLogger(__name__)

from graph_rewrite import draw

from spannerlib.utils import checkLogs,serialize_df_values,serialize_graph,get_new_node_name
from spannerlib.span import Span
from spannerlib.ra import equalConstTheta,equalColTheta
from spannerlib.data_types import (
    _infer_relation_schema,
    Var,
    FreeVar,
    RelationDefinition,
    Relation,
    IEFunction,
    IERelation,
    Rule,
    pretty,
)

Here we have functions for manipulating rules into term graphs

In [ ]:
# TODO, redo the rule to term graph to have:
# a relation to graph function with the select and project stuff
# an ie relation to graph, that adds the new ie function idea



In [ ]:
# TODO from here

### Adding constant terms to term graph

In [ ]:
#| export
def add_select_constants(g,source,terms):
    """
    adds a select node as a father to source, with the constant terms defined in terms
    if no constant terms are defined, does nothing
    returns the select node if it was added or the source node if not


    Example - R(X,3):
    source <- select(theta(col1=3))
    """
    need_select = any(not isinstance(term,FreeVar) for term in terms)
    if not need_select:
        return source
    
    select_pos_val = list()
    for i,term in enumerate(terms):
        if not isinstance(term,FreeVar):
            select_pos_val.append((i,term))
    
    select_name = get_new_node_name(g)
    g.add_node(select_name, op='select',theta=equalConstTheta(*select_pos_val))
    g.add_edge(select_name,source)
    return select_name

def add_select_col_eq(g,source,terms):
    """
    add nodes to filter source to include equality constraints between columns with the same free var
    if no such constraints are defined, does nothing 
    returns the top most node if it was added or the source node if not

    Example - R(X,Y,X):
    source <- select(theta(col1=col3))
    return project
    """
    
    # compute first position of each free var and store duplicates positions
    first_positions = {}
    duplicate_positions = defaultdict(list)
    for i,term in enumerate(terms):
        if isinstance(term,FreeVar):
            if term.name in first_positions:
                duplicate_positions[term.name].append(i)
            else:
                first_positions[term.name] = i

    # if no duplicates do nothing
    if len(duplicate_positions) == 0:
        return source

    # add equality constraints
    equality_constraints = []
    for duplicate_var,positions in duplicate_positions.items():
        i = first_positions[duplicate_var]
        for j in positions:
            equality_constraints.append((i,j))


    # override name so that select node remains the topmost node
    select_node = get_new_node_name(g)
    g.add_node(select_node, op='select',theta=equalColTheta(*equality_constraints))
    g.add_edge(select_node,source)

    return select_node


def add_project_uniq_free_vars(g,source,terms):
    """
    add a rename and a project node to leave only the first appearance of each free var in a relation
    returns the project node if it was added or the source node if not

    Example - R(X,Y,X,3):
    source <- rename(names=[(1,X),(2,Y),(3,_F3),(4,_F4)]) <- project(on=[X,Y])
    return project
    """
    seen_vars = []
    uniq_names = []
    for i,term in enumerate(terms):
        if isinstance(term,FreeVar) and term.name not in seen_vars:
            seen_vars.append(term.name)
            uniq_names.append(term.name)
        else:
            uniq_names.append(f'_F{i}')

    top_node = source
    # rename  to have unique names
    rename_node = get_new_node_name(g)
    g.add_node(rename_node, op='rename',names=list(enumerate(uniq_names)))
    g.add_edge(rename_node,top_node)
    top_node = rename_node
    # project to keep only the first appearance of each free var
    project_node = get_new_node_name(g)
    g.add_node(project_node, op='project',on=list(seen_vars))
    g.add_edge(project_node,top_node)

    return project_node

def add_product_constants(g,source,terms):
    """
    adds a product node as a father to source, with the constant terms defined in terms
    if no constant terms are defined, does nothing
    returns the product node if it was added or the source not if not

    Example - F(X,3,Y,4)->(Z):
    source                       <- product() <- project(on=[X,_C1,Y,_C2])
    get_const({'_C1':3,'_C2':4}) <- 

    example 2 - F(3,4)->(Z):
        get_const({'_C1':3,'_C2':4})


    """
    has_consts = any(not isinstance(term,FreeVar) for term in terms)
    has_vars = any(isinstance(term,FreeVar) for term in terms)
    if has_vars and not has_consts:
        return source
    

    product_pos_val = list()
    for i,term in enumerate(terms):
        if not isinstance(term,FreeVar):
            product_pos_val.append((i,term))
    
    const_dict = {f'_C{i}':val for i,val in product_pos_val}

    const_node = get_new_node_name(g)
    g.add_node(const_node, op='get_const',const_dict = const_dict)

    if has_consts and not has_vars:
        return const_node


    product_node = get_new_node_name(g)
    g.add_node(product_node, op='product')
    g.add_edge(product_node,source)
    g.add_edge(product_node,const_node)

    project_node = get_new_node_name(g)
    project_order = []
    for i,term in enumerate(terms):
        if isinstance(term,FreeVar):
            project_order.append(term.name)
        else:
            project_order.append(f'_C{i}')
    g.add_node(project_node, op='project',on=project_order)
    g.add_edge(project_node,product_node)
    
    return project_node



#### Tests

In [ ]:
# constants
g = nx.DiGraph()
g.add_node('A')

add_select_constants(g,'A',[1,FreeVar(name='X')])
draw(g)

# equal free vars
g = nx.DiGraph()
g.add_node('A')

add_select_col_eq(g,'A',[FreeVar(name='X'),FreeVar(name='Y'),FreeVar(name='X'),FreeVar(name='X')])
draw(g)

# project uniq free vars - no need 
g = nx.DiGraph()
g.add_node('A')

add_project_uniq_free_vars(g,'A',[FreeVar(name='X'),FreeVar(name='Y')])
draw(g)

# project uniq free vars - constants 
g = nx.DiGraph()
g.add_node('A')

add_project_uniq_free_vars(g,'A',[FreeVar(name='X'),FreeVar(name='Y'),2])
draw(g)

# project uniq free vars
g = nx.DiGraph()
g.add_node('A')

add_project_uniq_free_vars(g,'A',[FreeVar(name='X'),FreeVar(name='Y'),FreeVar(name='X'),1])
draw(g)

g = nx.DiGraph()
g.add_node('A')

add_product_constants(g,'A',[FreeVar(name='X'),1,FreeVar(name='Y'),2])
draw(g)

g = nx.DiGraph()

add_product_constants(g,None,[1,2,3])
draw(g)

### Adding body relations

In [ ]:
#| export
def mask_terms(terms,mask):
    if mask is None:
        return terms
    if not len(mask) == len(terms):
        raise ValueError(f'mask_constant_select should have the same length as the relation terms, got terms={terms},mask={mask}')
    return [term if not mask[i] else FreeVar(name='_F{i}') for i,term in enumerate(terms)]

def add_relation(g,terms,name=None,source=None,mask_constant_select=None):
    """
    adds a relation to the graph
    WLOG a relation of the form R(X,Y,const)
    should be of the abstract form get(R)<-rename(0:X,1:Y)<-select(2:const)<-project(on=[X,Y])
    if source is not None, source replaces get(R)
    returns (top most node, bottom most node) 
    """
    if source is None:
        g.add_node(name,rel=name)
        source = name
        rename_node = get_new_node_name(g)
        g.add_node(rename_node, op='rename',names=[(i,term.name) for i,term in enumerate(terms) if isinstance(term,FreeVar)])
        g.add_edge(rename_node,source)
        source = rename_node

    # select on constant terms
    select_constant_terms = mask_terms(terms,mask_constant_select)
    select_node = add_select_constants(g,source,select_constant_terms)

    # add select on equal free vars assignments for freevars with the same name
    select_node = add_select_col_eq(g,select_node,terms)

    project_node = add_project_uniq_free_vars(g,select_node,terms)
    
    return (project_node,source)
    

def add_ie_relation(g,rel):
    """
    adds an ie relation to the graph
    WLOG a relation of the form f(X,Y,c1)->(Z,X,c2)
    should be of the abstract form 
    project(X,Y)          <- product()<-projecton=[X,Y,_C2]<-ie_map(f)<-select(col_5==c2)<-select(col_0==col_4)<-rename(0:X,1:Y,3:Z)<-project(on=[X,Y,Z])
    get_const({'_C2':c1}) <-
    returns (top most node, bottom most node)
    """
    ie_has_variable_inputs = any(isinstance(term,FreeVar) for term in rel.in_terms)
    if ie_has_variable_inputs:
        project_input_vars = get_new_node_name(g)
        g.add_node(project_input_vars, op='project',on=[term.name for term in rel.in_terms if isinstance(term,FreeVar)])
    else:
        project_input_vars = None
    product_node = add_product_constants(g,project_input_vars,rel.in_terms)
    ie_map_node = get_new_node_name(g)
    g.add_node(ie_map_node, op='ie_map',func=rel.name,in_arity=len(rel.in_terms),out_arity=len(rel.out_terms))
    g.add_edge(ie_map_node,product_node)
    # we will get combined input+output relation from the ie map so now we reason based on it

    combined_terms = rel.in_terms+rel.out_terms
    # TODO add a naive rename node to rename the output variables so we can join input and output
    rename_node = get_new_node_name(g)
    g.add_node(rename_node, op='rename',names=[(i,term.name) for i,term in enumerate(combined_terms) if isinstance(term,FreeVar)])
    g.add_edge(rename_node,ie_map_node)

    
    # we only need to select on the output constants, since the input constants where generated via the product
    # so we mask the input terms
    mask_input_constants = [True]*len(rel.in_terms)+[False]*len(rel.out_terms)
    top_node,_ = add_relation(g,terms=combined_terms,source=rename_node,mask_constant_select=mask_input_constants)

    return top_node,project_input_vars


#### Tests

In [ ]:
g=nx.DiGraph()
add_relation(g,name='R',terms=[FreeVar(name='X'),FreeVar(name='Y'),1])
draw(g)
serialize_graph(g)
assert serialize_graph(g) == ([('R', {'rel': 'R'}),
  (0, {'op': 'rename', 'names': [(0, 'X'), (1, 'Y')]}),
  (1, {'op': 'select', 'theta': equalConstTheta((2,1))}),
  (2, {'op': 'rename', 'names': [(0, 'X'), (1, 'Y'), (2, '_F2')]}),
  (3, {'op': 'project', 'on': ['X', 'Y']})],
 [(0, 'R', {}), (1, 0, {}), (2, 1, {}), (3, 2, {})])

# TODO add cases with free vars with the same name

g=nx.DiGraph()
add_relation(g,name='R',terms=[FreeVar(name='X'),FreeVar(name='Y')])
draw(g)
serialize_graph(g)
assert serialize_graph(g) == ([('R', {'rel': 'R'}), (0, {'op': 'rename', 'names': [(0, 'X'), (1, 'Y')]})],
 [(0, 'R', {})])


g=nx.DiGraph()
add_relation(g,name='R',terms=[FreeVar(name='X'),3,FreeVar(name='Y'),FreeVar(name='X'),FreeVar(name='X'),FreeVar(name='Y'),2])
draw(g)
assert serialize_graph(g) == ([('R', {'rel': 'R'}),
  (0,
   {'op': 'rename',
    'names': [(0, 'X'), (2, 'Y'), (3, 'X'), (4, 'X'), (5, 'Y')]}),
  (1, {'op': 'select', 'theta': equalConstTheta((1,3),(6,2))}),
  (2, {'op': 'select', 'theta': equalColTheta((0,3),(0,4),(2,5))}),
  (3,
   {'op': 'rename', 'names': [(0, 'X'),(1, '_F1'),(2, 'Y'),(3, '_F3'),(4, '_F4'),(5, '_F5'),(6, '_F6')]}),
  (4, {'op': 'project', 'on': ['X', 'Y']})],
 [(0, 'R', {}), (1, 0, {}), (2, 1, {}), (3, 2, {}), (4, 3, {})])


In [ ]:
g=nx.DiGraph()
add_ie_relation(g,IERelation(name='f',in_terms=[FreeVar(name='X'),FreeVar(name='Y'),1],out_terms=[FreeVar(name='Z'),FreeVar(name='X'),2]))
draw(g)
# assert serialize_graph(g) == ([(0, {'op': 'project', 'on': ['X', 'Y']}),
#   (1, {'op': 'get_const', 'const_dict': {'_C2': 1}}),
#   (2, {'op': 'product'}),
#   (3, {'op': 'project', 'on': ['X', 'Y', '_C2']}),
#   (4,
#    {'op': 'ie_map',
#     'func': 'f',
#     'in_arity': 3,
#     'out_arity':3}),
#   (5, {'op': 'select', 'theta': equalConstTheta((5, 2))}),
#   (6, {'op': 'select', 'theta': equalColTheta((0,4))}),
#   (7, {'op': 'rename', 'names': [(0, 'X'),
#      (1, 'Y'),
#      (2, '_F2'),
#      (3, 'Z'),
#      (4, '_F4'),
#      (5, '_F5')]}),
#   (8, {'op': 'project', 'on': ['X', 'Y', 'Z']})],
#  [(2, 0, {}),
#   (2, 1, {}),
#   (3, 2, {}),
#   (4, 3, {}),
#   (5, 4, {}),
#   (6, 5, {}),
#   (7, 6, {}),
#   (8, 7, {})])

In [ ]:
g=nx.DiGraph()
top,bottom = add_ie_relation(g,IERelation(name='f',in_terms=[1,2,3],out_terms=[FreeVar(name='Z'),FreeVar(name='X'),2]))
draw(g)
serialize_graph(g) == ([(0, {'op': 'get_const', 'const_dict': {'_C0': 1, '_C1': 2, '_C2': 3}}),
  (1, {'op': 'ie_map', 'func': 'f', 'in_arity': 3,'out_arity':3}),
  (2, {'op': 'select', 'theta': equalConstTheta((5,2))}),
  (3, {'op': 'rename', 'names': [(3, 'Z'), (4, 'X')]}),
  (4, {'op': 'project', 'on': ['Z', 'X']})],
 [(1, 0, {}), (2, 1, {}), (3, 2, {}), (4, 3, {})])
top,bottom

(5, None)

### Computing bounding order of rules

In [ ]:
#| export
def get_bounding_order(rule:Rule):
    """Get an order of evaluation for the body of a rule
    this is a very naive ordering that can be heavily optimized"""

    # we start with all relations since they can be bound at once
    order = list()
    bounded_vars = set()
    for rel in rule.body:
        if isinstance(rel,Relation):
            order.append(rel)
            for term in rel.terms:
                if isinstance(term,FreeVar):
                    bounded_vars.add(term)

    unordered_ierelations = {rel for rel in rule.body if isinstance(rel,IERelation)}
    while len(unordered_ierelations) > 0:
        for ie_rel in unordered_ierelations:
            in_free_vars = {term for term in ie_rel.in_terms if isinstance(term,FreeVar)}
            if in_free_vars.issubset(bounded_vars):
                order.append(ie_rel)
                out_free_vars = {term for term in ie_rel.out_terms if isinstance(term,FreeVar)}
                bounded_vars = bounded_vars.union(out_free_vars)
                unordered_ierelations.remove(ie_rel)
                break

    return order

In [ ]:
r = Rule(
    head=Relation(name='R', terms=[FreeVar(name='X'), FreeVar(name='Y'), FreeVar(name='Z')]),
    body=[
        IERelation(name='T2', in_terms=[FreeVar(name='X'), FreeVar(name='Y')], out_terms=[FreeVar(name='W'), FreeVar(name='Z')]),
        IERelation(name='T', in_terms=[FreeVar(name='X'), 1], out_terms=[FreeVar(name='Y'), FreeVar(name='Z')]),
        Relation(name='S', terms=[FreeVar(name='X'), Span(start=1,end=4)]),
        Relation(name='S2', terms=[FreeVar(name='X'), FreeVar(name='A'),FreeVar(name='B')]),

    ])

print(pretty(r))

order = get_bounding_order(r)
assert [o.name for o in order ] == ['S','S2', 'T', 'T2']
order

R(X,Y,Z) <- T2(X,Y) -> (W,Z),T(X,1) -> (Y,Z),S(X,[1,4)),S2(X,A,B)


[Relation(name='S', terms=[FreeVar(name='X'), [1,4)]),
 Relation(name='S2', terms=[FreeVar(name='X'), FreeVar(name='A'), FreeVar(name='B')]),
 IERelation(name='T', in_terms=[FreeVar(name='X'), 1], out_terms=[FreeVar(name='Y'), FreeVar(name='Z')]),
 IERelation(name='T2', in_terms=[FreeVar(name='X'), FreeVar(name='Y')], out_terms=[FreeVar(name='W'), FreeVar(name='Z')])]

### Adding rules 

In [ ]:
#| export
def rule_to_graph(rule:Rule,rule_id):
    """
    converts a rule to a graph
    """
    g=nx.DiGraph()
    body_rels = get_bounding_order(rule)

    top_bottom_nodes = []

    # add all body relations to the graph
    for rel in body_rels:
        if isinstance(rel,Relation):
            top,bottom = add_relation(g,name=rel.name,terms=rel.terms)
            top_bottom_nodes.append((top,bottom))
        elif isinstance(rel,IERelation):
            top,bottom = add_ie_relation(g,rel)
            top_bottom_nodes.append((top,bottom))

    # connect outputs of different rels via joins
    # and connect input of ie functons into the join
    for i,((top,bottom),rel) in enumerate(zip(top_bottom_nodes,body_rels)):
        logger.debug(f'connecting bodies iteration {i}')
        if i == 0:
            prev_top = top
            continue
        logger.debug(f'connecting {prev_top} to {top}')

        join_node_name = get_new_node_name(g)
        g.add_node(join_node_name, op='join')
        g.add_edge(join_node_name,prev_top)
        g.add_edge(join_node_name,top)

        if isinstance(rel,IERelation):
            ie_has_variable_inputs = any(isinstance(term,FreeVar) for term in rel.in_terms)
            if ie_has_variable_inputs:
                logger.debug(f'adding input of ie {bottom} to join {prev_top}')
                g.add_edge(bottom,prev_top)
        prev_top = join_node_name

    # project all assignments into the head
    head_project_name = get_new_node_name(g)
    g.add_node(head_project_name, op='project', on=[term.name for term in rule.head.terms],rel=f'_{rule.head.name}_{rule_id}')
    g.add_edge(head_project_name,prev_top)

    # add a union for each rule for the given head
    g.add_node(rule.head.name,op='union',rel=rule.head.name)
    g.add_edge(rule.head.name,head_project_name)

    # add rule id for each node
    for u in g.nodes:
        g.nodes[u]['rule_id'] = {rule_id}
    return g


#### Test

In [ ]:
const_ie_rule = Rule(
    head=Relation(name='R', terms=[FreeVar(name='X')]),
    body=[
        IERelation(name='T', in_terms=[1, 2, 3], out_terms=[FreeVar(name='X')]),
        
    ])
print(pretty(const_ie_rule))

const_ie_rule2 = Rule(
    head=Relation(name='R', terms=[FreeVar(name='X')]),
    body=[
        Relation(name='S', terms=[FreeVar(name='X')]),
        IERelation(name='T', in_terms=[1, 2, 3], out_terms=[FreeVar(name='X')]),
        
    ])
print(pretty(const_ie_rule2))


R(X) <- T(1,2,3) -> (X)
R(X) <- S(X),T(1,2,3) -> (X)


In [ ]:
const_ie_rule2

Rule(head=Relation(name='R', terms=[FreeVar(name='X')]), body=[Relation(name='S', terms=[FreeVar(name='X')]), IERelation(name='T', in_terms=[1, 2, 3], out_terms=[FreeVar(name='X')])])

In [ ]:
with checkLogs():
  g = rule_to_graph(const_ie_rule2,0)
draw(g)
serialize_graph(g)


__main__ - DEBUG - connecting bodies iteration 0
__main__ - DEBUG - connecting bodies iteration 1
__main__ - DEBUG - connecting 0 to 5


([('S', {'rel': 'S', 'rule_id': {0}}),
  (0, {'op': 'rename', 'names': [(0, 'X')], 'rule_id': {0}}),
  (1,
   {'op': 'get_const',
    'const_dict': {'_C0': 1, '_C1': 2, '_C2': 3},
    'rule_id': {0}}),
  (2,
   {'op': 'ie_map',
    'func': 'T',
    'in_arity': 3,
    'out_arity': 1,
    'rule_id': {0}}),
  (3, {'op': 'rename', 'names': [(3, 'X')], 'rule_id': {0}}),
  (4,
   {'op': 'rename',
    'names': [(0, '_F0'), (1, '_F1'), (2, '_F2'), (3, 'X')],
    'rule_id': {0}}),
  (5, {'op': 'project', 'on': ['X'], 'rule_id': {0}}),
  (6, {'op': 'join', 'rule_id': {0}}),
  (7, {'op': 'project', 'on': ['X'], 'rel': '_R_0', 'rule_id': {0}}),
  ('R', {'op': 'union', 'rel': 'R', 'rule_id': {0}})],
 [(0, 'S', {}),
  (2, 1, {}),
  (3, 2, {}),
  (4, 3, {}),
  (5, 4, {}),
  (6, 0, {}),
  (6, 5, {}),
  (7, 6, {}),
  ('R', 7, {})])

In [ ]:
g = rule_to_graph(const_ie_rule,0)
draw(g)
# assert serialize_graph(g) == ([(0,
#    {'op': 'get_const',
#     'const_dict': {'_C0': 1, '_C1': 2, '_C2': 3},
#     'rule_id': {0}}),
#   (1, {'op': 'ie_map', 'func': 'T', 'in_arity': 3,'out_arity':1, 'rule_id': {0}}),
#   (2, {'op': 'rename', 'names': [(3, 'X')], 'rule_id': {0}}),
#   (3, {'op': 'project', 'on': ['X'], 'rule_id': {0}}),
#   (4, {'op': 'project', 'on': ['X'], 'rel': '_R_0', 'rule_id': {0}}),
#   ('R', {'op': 'union', 'rel': 'R', 'rule_id': {0}})],
#  [(1, 0, {}), (2, 1, {}), (3, 2, {}), (4, 3, {}), ('R', 4, {})])

In [ ]:
g = rule_to_graph(r,0)
print(pretty(r))
draw(g)
# assert serialize_graph(g) == ([('S', {'rel': 'S', 'rule_id': {0}}),
#   (0, {'op': 'rename', 'names': [(0, 'X')], 'rule_id': {0}}),
#   (1, {'op': 'select', 'theta': equalConstTheta((1,Span(1,4))), 'rule_id': {0}}),
#   (2, {'op': 'project', 'on': ['X'], 'rule_id': {0}}),
#   ('S2', {'rel': 'S2', 'rule_id': {0}}),
#   (3,
#    {'op': 'rename', 'names': [(0, 'X'), (1, 'A'), (2, 'B')], 'rule_id': {0}}),
#   (4, {'op': 'project', 'on': ['X'], 'rule_id': {0}}),
#   (5, {'op': 'get_const', 'const_dict': {'_C1': 1}, 'rule_id': {0}}),
#   (6, {'op': 'product', 'rule_id': {0}}),
#   (7, {'op': 'project', 'on': ['X', '_C1'], 'rule_id': {0}}),
#   (8, {'op': 'ie_map', 'func': 'T', 'rule_id': {0},'in_arity': 2,'out_arity':2}),
#   (9,
#    {'op': 'rename', 'names': [(0, 'X'), (2, 'Y'), (3, 'Z')], 'rule_id': {0}}),
#   (10, {'op': 'project', 'on': ['X', 'Y', 'Z'], 'rule_id': {0}}),
#   (11, {'op': 'project', 'on': ['X', 'Y'], 'rule_id': {0}}),
#   (12, {'op': 'ie_map', 'func': 'T2', 'rule_id': {0},'in_arity': 2,'out_arity':2}),
#   (13,
#    {'op': 'rename',
#     'names': [(0, 'X'), (1, 'Y'), (2, 'W'), (3, 'Z')],
#     'rule_id': {0}}),
#   (14, {'op': 'project', 'on': ['X', 'Y', 'W', 'Z'], 'rule_id': {0}}),
#   (15, {'op': 'join', 'rule_id': {0}}),
#   (16, {'op': 'join', 'rule_id': {0}}),
#   (17, {'op': 'join', 'rule_id': {0}}),
#   (18,
#    {'op': 'project', 'on': ['X', 'Y', 'Z'], 'rel': '_R_0', 'rule_id': {0}}),
#   ('R', {'op': 'union', 'rel': 'R', 'rule_id': {0}})],
#  [(0, 'S', {}),
#   (1, 0, {}),
#   (2, 1, {}),
#   (3, 'S2', {}),
#   (4, 15, {}),
#   (6, 4, {}),
#   (6, 5, {}),
#   (7, 6, {}),
#   (8, 7, {}),
#   (9, 8, {}),
#   (10, 9, {}),
#   (11, 16, {}),
#   (12, 11, {}),
#   (13, 12, {}),
#   (14, 13, {}),
#   (15, 2, {}),
#   (15, 3, {}),
#   (16, 15, {}),
#   (16, 10, {}),
#   (17, 16, {}),
#   (17, 14, {}),
#   (18, 17, {}),
#   ('R', 18, {})])

R(X,Y,Z) <- T2(X,Y) -> (W,Z),T(X,1) -> (Y,Z),S(X,[1,4)),S2(X,A,B)


### Compose term graphs

In [ ]:
#| export
def graph_compose(g1,g2,mapping_dict,debug=False):
    """compose two graphs with a mapping dict"""
    # if there is a node in g2 that is renamed but has a name collision with an existing node that is not renamed, we will rename the existing node to a uniq name
    # making new names into a digraph is a dirty hack, TODO resolve this
    save_new_names= nx.DiGraph()
    original_mapping_dict = mapping_dict.copy()
    for u2 in g2.nodes():
        if u2 not in mapping_dict and u2 in g1.nodes():
            mapping_dict[u2] = get_new_node_name(g2,avoid_names_from=[g1,save_new_names])
            save_new_names.add_node(mapping_dict[u2])
    if debug:
        return mapping_dict
    g2 = nx.relabel_nodes(g2,mapping_dict,copy=True)

    merged_graph = nx.compose(g1,g2)
    for old_name,new_name in original_mapping_dict.items():
        rule_ids1 = g1.nodes[old_name].get('rule_id',set())
        rule_ids2 = g2.nodes[new_name].get('rule_id',set())
        merged_rule_ids = rule_ids1.union(rule_ids2)
        merged_graph.nodes[new_name]['rule_id'] = merged_rule_ids



    return merged_graph


#### Test

In [ ]:
r1 = Rule(
    head=Relation(name='R', terms=[FreeVar(name='X'), FreeVar(name='Y')]),
    body=[
        Relation(name='S', terms=[FreeVar(name='X'),FreeVar(name='Y')]),
        Relation(name='S2', terms=[FreeVar(name='X'), FreeVar(name='A'),1]),
    ])

r2 = Rule(
    head=Relation(name='R', terms=[FreeVar(name='X'), FreeVar(name='Y')]),
    body=[
        Relation(name='S', terms=[FreeVar(name='X'),FreeVar(name='Y')]),
    ])

r3 = Rule(
    head=Relation(name='R2', terms=[FreeVar(name='X'), FreeVar(name='Y')]),
    body=[
        Relation(name='S3', terms=[FreeVar(name='X'),FreeVar(name='Y')]),
        Relation(name='S2', terms=[FreeVar(name='X'), FreeVar(name='A'),1]),
    ])
rules = [r1,r2,r3]
for r in rules:
    print(pretty(r))
t1,t2,t3 = [rule_to_graph(r,i) for i,r in enumerate(rules)]

R(X,Y) <- S(X,Y),S2(X,A,1)
R(X,Y) <- S(X,Y)
R2(X,Y) <- S3(X,Y),S2(X,A,1)


In [ ]:
draw(t1)
draw(t2)
draw(t3)

In [ ]:
m= graph_compose(t1,t2,
    mapping_dict = {'S':'S','R':'R',0:0}) 
draw(m)

### Merge term graphs

In [ ]:
#| export
def merge_term_graphs_pair(g1,g2,exclude_props = ['label'],debug=False):
    """merge two term graphs into one term graph
    when talking about term graphs, 2 nodes if their data is identical and all of their children are identical
    but we would also like to merge rules for the same head, so we will also nodes that have the same 'rel' attribute
    """

    def _are_nodes_equal(g1,u1,g2,u2):

        u1_data = g1.nodes[u1]
        u2_data = g2.nodes[u2]
        
        if 'rel' in u1_data and 'rel' in u2_data:
            return u1_data['rel'] == u2_data['rel']


        
        return False
        # TODO this old code tries to merge nodes, but then its hard to remember which belong to which rules so we only merge
        # so we will do this merging per query
        u1_clean_data = {k:v for k,v in u1_data.items() if k not in exclude_props}
        u2_clean_data = {k:v for k,v in u2_data.items() if k not in exclude_props}

        are_equal = u1_clean_data == u2_clean_data and all(v2 in node_mappings for v2 in g2.successors(u2))
        return are_equal
        

    # we will check for each node in g2 if it has a node in g1 which is it's equal.
    # and save that in a mapping
    node_mappings=dict()# g2 node name to g1 node name
    # we use the fact that g2 is going to be acyclic to travers it in postorder
    for u2 in nx.dfs_postorder_nodes(g2):
        for u1 in g1.nodes():
            if _are_nodes_equal(g1,u1,g2,u2):
                node_mappings[u2] = u1
                break



    if debug:
        return node_mappings
    else:
        return graph_compose(g1,g2,node_mappings)



def merge_term_graphs(gs,exclude_props = ['label'],debug=False):
    """merge a list of term graphs into one term graph
    """
    merge = gs[0]
    for g in gs[1:-1]:
        merge = merge_term_graphs_pair(merge,g,exclude_props,debug=False)
    # if debug, we run debug only on the last merge so we can iteratively debug a list of merges
    return merge_term_graphs_pair(merge,gs[-1],exclude_props,debug=debug)


#### Tests

In [ ]:
r1 = Rule(
    head=Relation(name='A', terms=[FreeVar(name='X'), FreeVar(name='Y')]),
    body=[
        Relation(name='B', terms=[FreeVar(name='X'),FreeVar(name='Y')]),
    ])

r2 = Rule(
    head=Relation(name='A', terms=[FreeVar(name='X'), FreeVar(name='Y')]),
    body=[
        Relation(name='C', terms=[FreeVar(name='X'),FreeVar(name='Y')]),
    ])

r3 = Rule(
    head=Relation(name='B', terms=[FreeVar(name='X'), FreeVar(name='Y')]),
    body=[
        Relation(name='D', terms=[FreeVar(name='X'),FreeVar(name='Y')]),
    ])

r4 = Rule(
    head=Relation(name='B', terms=[FreeVar(name='X'), FreeVar(name='Y')]),
    body=[
        Relation(name='A', terms=[FreeVar(name='X'),FreeVar(name='Y')]),
    ])



In [ ]:
rules = [r1,r2,r3,r4]
print([pretty(r) for r in rules])
for r in rules:
    draw(rule_to_graph(r,0))


['A(X,Y) <- B(X,Y)', 'A(X,Y) <- C(X,Y)', 'B(X,Y) <- D(X,Y)', 'B(X,Y) <- A(X,Y)']


In [ ]:
m = merge_term_graphs([rule_to_graph(r,i) for i,r in enumerate(rules)])
draw(m)
serialize_graph(m)
# assert serialize_graph(m) == ([('B', {'op': 'union', 'rel': 'B', 'rule_id': {0, 2, 3}}),
#   (0, {'op': 'rename', 'names': [(0, 'X'), (1, 'Y')], 'rule_id': {0}}),
#   (1, {'op': 'project', 'on': ['X', 'Y'], 'rel': '_A_0', 'rule_id': {0}}),
#   ('A', { 'rel': 'A', 'rule_id': {0, 1, 3}}),
#   ('C', { 'rel': 'C', 'rule_id': {1}}),
#   (2, {'op': 'rename', 'names': [(0, 'X'), (1, 'Y')], 'rule_id': {1}}),
#   (3, {'op': 'project', 'on': ['X', 'Y'], 'rel': '_A_1', 'rule_id': {1}}),
#   ('D', { 'rel': 'D', 'rule_id': {2}}),
#   (4, {'op': 'rename', 'names': [(0, 'X'), (1, 'Y')], 'rule_id': {2}}),
#   (5, {'op': 'project', 'on': ['X', 'Y'], 'rel': '_B_2', 'rule_id': {2}}),
#   (6, {'op': 'rename', 'names': [(0, 'X'), (1, 'Y')], 'rule_id': {3}}),
#   (7, {'op': 'project', 'on': ['X', 'Y'], 'rel': '_B_3', 'rule_id': {3}})],
#  [('B', 5, {}),
#   ('B', 7, {}),
#   (0, 'B', {}),
#   (1, 0, {}),
#   ('A', 1, {}),
#   ('A', 3, {}),
#   (2, 'C', {}),
#   (3, 2, {}),
#   (4, 'D', {}),
#   (5, 4, {}),
#   (6, 'A', {}),
#   (7, 6, {})])

([('B', {'rel': 'B', 'rule_id': {0, 2, 3}, 'op': 'union'}),
  (0, {'op': 'rename', 'names': [(0, 'X'), (1, 'Y')], 'rule_id': {0}}),
  (1, {'op': 'project', 'on': ['X', 'Y'], 'rel': '_A_0', 'rule_id': {0}}),
  ('A', {'op': 'union', 'rel': 'A', 'rule_id': {0, 1, 3}}),
  ('C', {'rel': 'C', 'rule_id': {1}}),
  (2, {'op': 'rename', 'names': [(0, 'X'), (1, 'Y')], 'rule_id': {1}}),
  (3, {'op': 'project', 'on': ['X', 'Y'], 'rel': '_A_1', 'rule_id': {1}}),
  ('D', {'rel': 'D', 'rule_id': {2}}),
  (4, {'op': 'rename', 'names': [(0, 'X'), (1, 'Y')], 'rule_id': {2}}),
  (5, {'op': 'project', 'on': ['X', 'Y'], 'rel': '_B_2', 'rule_id': {2}}),
  (6, {'op': 'rename', 'names': [(0, 'X'), (1, 'Y')], 'rule_id': {3}}),
  (7, {'op': 'project', 'on': ['X', 'Y'], 'rel': '_B_3', 'rule_id': {3}})],
 [('B', 5, {}),
  ('B', 7, {}),
  (0, 'B', {}),
  (1, 0, {}),
  ('A', 1, {}),
  ('A', 3, {}),
  (2, 'C', {}),
  (3, 2, {}),
  (4, 'D', {}),
  (5, 4, {}),
  (6, 'A', {}),
  (7, 6, {})])

In [ ]:
m = merge_term_graphs([t1,t2])
draw(m)
# assert serialize_graph(m) == ([('S', { 'rel': 'S', 'rule_id': {0, 1}}),
#   (0, {'op': 'rename', 'names': [(0, 'X'), (1, 'Y')], 'rule_id': {0}}),
#   ('S2', { 'rel': 'S2', 'rule_id': {0}}),
#   (1, {'op': 'rename', 'names': [(0, 'X'), (1, 'A')], 'rule_id': {0}}),
#   (2, {'op': 'select', 'theta': [(2, 1)], 'rule_id': {0}}),
#   (3, {'op': 'join', 'rule_id': {0}}),
#   (4, {'op': 'project', 'on': ['X', 'Y'], 'rel': '_R_0', 'rule_id': {0}}),
#   ('R', {'op': 'union', 'rel': 'R', 'rule_id': {0, 1}}),
#   (5, {'op': 'rename', 'names': [(0, 'X'), (1, 'Y')], 'rule_id': {1}}),
#   (6, {'op': 'project', 'on': ['X', 'Y'], 'rel': '_R_1', 'rule_id': {1}})],
#  [(0, 'S', {}),
#   (1, 'S2', {}),
#   (2, 1, {}),
#   (3, 0, {}),
#   (3, 2, {}),
#   (4, 3, {}),
#   ('R', 4, {}),
#   ('R', 6, {}),
#   (5, 'S', {}),
#   (6, 5, {})])

In [ ]:
m = merge_term_graphs([t1,t3])
draw(m)
# assert serialize_graph(m) == ([('S', { 'rel': 'S', 'rule_id': {0}}),
#   (0, {'op': 'rename', 'names': [(0, 'X'), (1, 'Y')], 'rule_id': {0}}),
#   ('S2', { 'rel': 'S2', 'rule_id': {0, 2}}),
#   (1, {'op': 'rename', 'names': [(0, 'X'), (1, 'A')], 'rule_id': {0}}),
#   (2, {'op': 'select', 'theta': [(2, 1)], 'rule_id': {0}}),
#   (3, {'op': 'join', 'rule_id': {0}}),
#   (4, {'op': 'project', 'on': ['X', 'Y'], 'rel': '_R_0', 'rule_id': {0}}),
#   ('R', {'op': 'union', 'rel': 'R', 'rule_id': {0}}),
#   ('S3', { 'rel': 'S3', 'rule_id': {2}}),
#   (5, {'op': 'rename', 'names': [(0, 'X'), (1, 'Y')], 'rule_id': {2}}),
#   (6, {'op': 'rename', 'names': [(0, 'X'), (1, 'A')], 'rule_id': {2}}),
#   (7, {'op': 'select', 'theta': [(2, 1)], 'rule_id': {2}}),
#   (8, {'op': 'join', 'rule_id': {2}}),
#   (9, {'op': 'project', 'on': ['X', 'Y'], 'rel': '_R2_2', 'rule_id': {2}}),
#   ('R2', {'op': 'union', 'rel': 'R2', 'rule_id': {2}})],
#  [(0, 'S', {}),
#   (1, 'S2', {}),
#   (2, 1, {}),
#   (3, 0, {}),
#   (3, 2, {}),
#   (4, 3, {}),
#   ('R', 4, {}),
#   (5, 'S3', {}),
#   (6, 'S2', {}),
#   (7, 6, {}),
#   (8, 5, {}),
#   (8, 7, {}),
#   (9, 8, {}),
#   ('R2', 9, {})])

In [ ]:
m = merge_term_graphs([t1,t2,t3])
draw(m)
# assert serialize_graph(m) == ([('S', { 'rel': 'S', 'rule_id': {0, 1}}),
#   (0, {'op': 'rename', 'names': [(0, 'X'), (1, 'Y')], 'rule_id': {0}}),
#   ('S2', { 'rel': 'S2', 'rule_id': {0, 2}}),
#   (1, {'op': 'rename', 'names': [(0, 'X'), (1, 'A')], 'rule_id': {0}}),
#   (2, {'op': 'select', 'theta': [(2, 1)], 'rule_id': {0}}),
#   (3, {'op': 'join', 'rule_id': {0}}),
#   (4, {'op': 'project', 'on': ['X', 'Y'], 'rel': '_R_0', 'rule_id': {0}}),
#   ('R', {'op': 'union', 'rel': 'R', 'rule_id': {0, 1}}),
#   (5, {'op': 'rename', 'names': [(0, 'X'), (1, 'Y')], 'rule_id': {1}}),
#   (6, {'op': 'project', 'on': ['X', 'Y'], 'rel': '_R_1', 'rule_id': {1}}),
#   ('S3', { 'rel': 'S3', 'rule_id': {2}}),
#   (7, {'op': 'rename', 'names': [(0, 'X'), (1, 'Y')], 'rule_id': {2}}),
#   (8, {'op': 'rename', 'names': [(0, 'X'), (1, 'A')], 'rule_id': {2}}),
#   (9, {'op': 'select', 'theta': [(2, 1)], 'rule_id': {2}}),
#   (10, {'op': 'join', 'rule_id': {2}}),
#   (11, {'op': 'project', 'on': ['X', 'Y'], 'rel': '_R2_2', 'rule_id': {2}}),
#   ('R2', {'op': 'union', 'rel': 'R2', 'rule_id': {2}})],
#  [(0, 'S', {}),
#   (1, 'S2', {}),
#   (2, 1, {}),
#   (3, 0, {}),
#   (3, 2, {}),
#   (4, 3, {}),
#   ('R', 4, {}),
#   ('R', 6, {}),
#   (5, 'S', {}),
#   (6, 5, {}),
#   (7, 'S3', {}),
#   (8, 'S2', {}),
#   (9, 8, {}),
#   (10, 7, {}),
#   (10, 9, {}),
#   (11, 10, {}),
#   ('R2', 11, {})])

In [ ]:
# TODO correct commented out tests once we see that the execution is correct

In [ ]:
#|hide
import nbdev; nbdev.nbdev_export()
     